In [1]:
!pip install nest-asyncio

In [1]:
!pip install openai python-multipart

In [1]:
import os

In [3]:
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, HTTPException
from fastapi.staticfiles import StaticFiles
from fastapi.responses import HTMLResponse
from typing import List, Dict, Optional, Any
from pydantic import BaseModel, ConfigDict
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from datetime import datetime
import json
import os
from dotenv import load_dotenv
import base64
from gtts import gTTS
import io
import uvicorn
import nest_asyncio
import uvicorn
from fastapi import File, UploadFile
import openai
import tempfile

# Load environment variables
load_dotenv()

class MenuItem(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    urun: str
    Fiyat: float
    ingredients: str
    category: str

class Order(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    items: List[Dict[str, any]] = []
    total: float = 0.0

class RestaurantRAG:
    def __init__(self, menu_path: str):
        """Initialize the RestaurantRAG system"""
        self.menu_path = menu_path
        self.menu_items = self.load_menu()
        self.vector_store = self.create_vector_store()
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer",
            input_key="human_input"
        )
        self.current_order = Order()
        self.setup_rag_chain()
        # Başlangıçta siparisler.txt'yi sil
        if os.path.exists("siparisler.txt"):
            os.remove("siparisler.txt")

    def load_menu(self) -> List[MenuItem]:
        """Load menu items from CSV file"""
        df = pd.read_csv(self.menu_path, delimiter=';')
        menu_items = []
        for _, row in df.iterrows():
            menu_items.append(
                MenuItem(
                    urun=row['urun'],
                    Fiyat=float(row['Fiyat']),
                    ingredients=row['ingredients'],
                    category=row['category']
                )
            )
        return menu_items

    def create_vector_store(self):
        """Create FAISS vector store from menu items"""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=400
        )
        
        menu_texts = []
        for item in self.menu_items:
            text = f"Category: {item.category}\nItem: {item.urun}\n"
            text += f"Price: {item.Fiyat} TL\nIngredients: {item.ingredients}\n"
            menu_texts.append(text)

        texts = text_splitter.create_documents(menu_texts)
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",
        chunk_size=8000)
        vector_store = FAISS.from_documents(texts,
        embeddings,
        normalize_L2=True )
        return vector_store

    def get_menu_by_category(self, category: Optional[str] = None) -> str:
        """Get menu items by category or full menu"""
        menu_text = ""
        if category:
            items = [item for item in self.menu_items if item.category.lower() == category.lower()]
            if not items:
                return f"Bu kategoride ürün bulunamadı: {category}"
            menu_text = f"\n{category.upper()} MENÜSÜ:\n"
            for item in items:
                menu_text += f"• {item.urun} - İçindekiler: {item.ingredients}\n"
        else:
            # Group items by category
            categories = {}
            for item in self.menu_items:
                if item.category not in categories:
                    categories[item.category] = []
                categories[item.category].append(item)
            
            menu_text = "\nTAM MENÜ:\n"
            for category, items in categories.items():
                menu_text += f"\n{category.upper()}:\n"
                for item in items:
                    menu_text += f"• {item.urun} - İçindekiler: {item.ingredients}\n"
        
        return menu_text

    

            
        return menu_text

    def setup_rag_chain(self):
        """Setup the RAG chain with custom prompt"""
        template = """You are a helpful and friendly Turkish restaurant waiter. Use the following information to help the customer.

        Menu Context:
        {context}

        Current Order Status:
        {current_order}

        Chat History:
        {chat_history}

        Customer: {question}
        {human_input}

   Instructions:
   YOUR NAME IS "MUTLU GARSON"
   Avoid providing incomplete information when answering. Never skip any items from the menu when the customer asks for it.
   If the customer asks for the menu, provide the full list of items (including all dish names and descriptions). Only include prices if the customer specifically requests them.
	1.	Help customers understand the menu and answer their questions naturally.
	2.	Provide the full menu when explicitly asked. Do not include prices unless specifically requested.
	3.	List menu items within a given price range when the customer provides one.
	4.	If asked about an unavailable item, inform the customer politely that it is not on the menu and suggest the closest available alternative.
	5.	Answer ingredient-related and pricing questions accurately when prompted.
	6.	Track and confirm the current order to avoid mistakes.
	7.	Handle special requests or modifications professionally, respecting the menu limitations.
	8.	Maintain a friendly and professional tone throughout the interaction.
	9.	Always respond in Turkish.

   Examples of behavior:
	•	When asked for the menu: Provide the full list of items without prices unless specifically requested.
	•	When asked for items in a price range: List all menu items that match the given range, mentioning their prices.
	•	When an unavailable item is requested: "Bu ürün menümüzde yok, ancak [en yakın alternatif ürün] öneririm."


        IMPORTANT FORMAT RULES:
        1. If the customer's message contains ANY indication of ordering, adding, or removing items, you MUST respond in this exact JSON format:
        {{"add_items": [{{"urun": "item_name", "quantity": number}}], "remove_items": [{{"urun": "item_name", "quantity": number}}], "response": "Your friendly Turkish response"}}
        
        2. For example, if customer says "2 tane pizza istiyorum":
        {{"add_items": [{{"urun": "Pizza", "quantity": 2}}], "remove_items": [], "response": "2 adet Pizza siparişinize eklendi. Başka bir arzunuz var mı?"}}
        
        3. If customer says "bir pizzayı iptal et":
        {{"add_items": [], "remove_items": [{{"urun": "Pizza", "quantity": 1}}], "response": "1 adet Pizza siparişinizden çıkarıldı. Başka bir isteğiniz var mı?"}}
    
        4. For regular questions about menu or general conversation, respond normally in Turkish without JSON format.


        Assistant:"""

        PROMPT = PromptTemplate(
            template=template,
            input_variables=["context", "question", "chat_history", "current_order", "human_input"]
        )

        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=ChatOpenAI(temperature=0.2, model="gpt-4o-mini",max_tokens=2000  ),
            retriever=self.vector_store.as_retriever(
                search_type="mmr",  # MMR search'e geçtik
            search_kwargs={
                "k": 30,  # k değerini arttırdık
                "fetch_k": 40,  # fetch_k ekledik
                "lambda_mult": 0.7 }
            ),
            memory=self.memory,
            combine_docs_chain_kwargs={"prompt": PROMPT},
            return_source_documents=True,
            chain_type="stuff",
            verbose=True
        )

    def _try_json_parsing(self, response_text: str) -> bool:
        """Try to parse JSON response for order actions"""
        try:
            # Find JSON content within the response
            json_str = ""
            start_idx = response_text.find('{')
            end_idx = response_text.rfind('}')
            
            if start_idx != -1 and end_idx != -1:
                json_str = response_text[start_idx:end_idx + 1]
                actions = json.loads(json_str)
                
                # Handle add items
                if "add_items" in actions and actions["add_items"]:
                    for item in actions["add_items"]:
                        if not isinstance(item, dict):
                            continue
                        self.add_to_order(item["urun"], item.get("quantity", 1))
                    return True
                
                # Handle remove items
                if "remove_items" in actions and actions["remove_items"]:
                    for item in actions["remove_items"]:
                        if not isinstance(item, dict):
                            continue
                        self.remove_from_order(item["urun"], item.get("quantity", 1))
                    return True
                
            return False
        
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            return False
        except Exception as e:
            print(f"Error processing order actions: {e}")
            return False



    def _parse_text_for_orders(self, text: str) -> None:
        """Parse plain text response for order items"""
        try:
            # Initialize items if needed
            if not hasattr(self.current_order, 'items'):
                self.current_order.items = []
                self.current_order.total = 0.0
    
            # Look for common patterns indicating orders
            lines = text.split('\n')
            for line in lines:
                if line.strip().startswith('-') or line.strip().startswith('•'):
                    # Extract item name
                    item_text = line.strip('- •').strip()
                    
                    # Look for quantity indicators
                    quantity = 1
                    if 'x' in item_text:
                        parts = item_text.split('x')
                        try:
                            quantity = int(parts[0].strip())
                            item_text = parts[1].strip()
                        except ValueError:
                            pass
    
                    # Find matching menu item
                    menu_item = next(
                        (item for item in self.menu_items 
                         if item.urun.lower() in item_text.lower()),
                        None
                    )
                    
                    if menu_item:
                        self.add_to_order(menu_item.urun, quantity)
    
        except Exception as e:
            print(f"Text parsing error: {str(e)}")



    def add_to_order(self, item_name: str, quantity: int = 1) -> str:
        """Add item to current order with enhanced initialization"""
        try:
            # Ensure current_order is properly initialized
            if not hasattr(self.current_order, 'items'):
                self.current_order.items = []
                self.current_order.total = 0.0
    
            # Find menu item
            menu_item = next(
                (item for item in self.menu_items if item.urun.lower() == item_name.lower()),
                None
            )
            
            if not menu_item:
                return f"Üzgünüm, {item_name} menümüzde bulunamadı."
    
            # Calculate new item total
            new_total = menu_item.Fiyat * quantity
            
            # Check for existing item
            existing_item = next(
                (item for item in self.current_order.items 
                 if item["urun"].lower() == item_name.lower()),
                None
            )
    
            if existing_item:
                existing_item["quantity"] += quantity
                existing_item["total"] = existing_item["Fiyat"] * existing_item["quantity"]
            else:
                # Add new item
                order_item = {
                    "urun": menu_item.urun,
                    "quantity": quantity,
                    "Fiyat": menu_item.Fiyat,
                    "total": new_total
                }
                self.current_order.items.append(order_item)

            # Update total
            self.current_order.total = sum(item["total"] for item in self.current_order.items)
            
            print(f"Added to order: {quantity}x {menu_item.urun}")
            print(f"Current order: {self.get_order_summary()}")
            
            return f"{quantity}x {menu_item.urun} siparişinize eklendi. Toplam tutar: {self.current_order.total:.2f} TL"
    
        except Exception as e:
            print(f"Error in add_to_order: {str(e)}")
            return f"Sipariş eklenirken bir hata oluştu: {str(e)}"

    def remove_from_order(self, item_name: str, quantity: Optional[int] = None) -> str:
        """Remove item from current order with improved error handling"""
        try:
            # Siparişte ürünü bul
            item_index = next(
                (idx for idx, item in enumerate(self.current_order.items)
                 if item["urun"].lower() == item_name.lower()),
                None
            )

            if item_index is None:
                return f"{item_name} mevcut siparişinizde bulunmuyor."

            item = self.current_order.items[item_index]
            
            if quantity is None or quantity >= item["quantity"]:
                # Ürünü tamamen çıkar
                self.current_order.total -= item["total"]
                self.current_order.items.pop(item_index)
                return f"{item['urun']} siparişinizden çıkarıldı."
            else:
                # Miktarı azalt
                item["quantity"] -= quantity
                old_total = item["total"]
                item["total"] = item["Fiyat"] * item["quantity"]
                self.current_order.total -= (old_total - item["total"])
                return f"{quantity}x {item['urun']} siparişinizden çıkarıldı."

        except Exception as e:
            print(f"Sipariş çıkarma hatası: {str(e)}")
            return f"Sipariş çıkarılırken bir hata oluştu: {str(e)}"

    def get_order_summary(self) -> str:
        """Get current order summary with improved formatting"""
        try:
            if not self.current_order.items:
                return "Henüz sipariş verilmedi."
            
            summary = "\n=== SİPARİŞ ÖZETİ ===\n"
            for item in self.current_order.items:
                summary += f"• {item['urun']} x {item['quantity']} = {item['total']:.2f} TL\n"
            summary += "=" * 20 + "\n"
            summary += f"TOPLAM TUTAR: {self.current_order.total:.2f} TL\n"
            summary += "=" * 20
            return summary

        except Exception as e:
            print(f"Sipariş özeti oluşturma hatası: {str(e)}")
            return "Sipariş özeti oluşturulurken bir hata oluştu."


    def save_order_to_file(self, order_details: str):
        """Save order details to siparisler.txt"""
        with open("siparisler.txt", "w", encoding="utf-8") as f:
            f.write("=== Sipariş Detayları ===\n")
            f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(order_details)
            f.write("\n=== Sipariş Sonu ===")

    async def process_message(self, message: str) -> Dict[str, Any]:
        """Process customer message and return response with audio"""
        try:
            inputs = {
                "question": message,
                "current_order": self.get_order_summary(),
                "chat_history": self.memory.load_memory_variables({})["chat_history"],
                "human_input": ""
            }
            
            response = await self.qa_chain.ainvoke(inputs)
            text_response = response["answer"]
    
            # JSON yanıtı işleme
            is_json_processed = self._try_json_parsing(text_response)
            
            if is_json_processed:
                try:
                    json_data = json.loads(text_response[text_response.find('{'):text_response.rfind('}')+1])
                    text_response = json_data.get("response", text_response)
                except:
                    pass





            # Generate speech from text response
            tts = gTTS(text=text_response, lang='tr')
            audio_io = io.BytesIO()
            tts.write_to_fp(audio_io)
            audio_io.seek(0)
            
            # Convert audio to base64
            audio_base64 = base64.b64encode(audio_io.read()).decode()
            
            order_data = {
            "items": self.current_order.items,
            "total": self.current_order.total
            }
    
            return {
                "text": text_response,
                "audio": audio_base64,
                "order": order_data  # Sipariş bilgilerini ekle
            }
            
        except Exception as e:
            return {
                "text": f"Üzgünüm, bir hata oluştu: {str(e)}",
                "audio": None,
                "order": None
            }


    

# FastAPI uygulaması
app = FastAPI()

# WebSocket bağlantı yöneticisi
class ConnectionManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []

    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)

    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)

    async def send_message(self, message: dict, websocket: WebSocket):
        await websocket.send_json(message)

manager = ConnectionManager()
restaurant_rag = RestaurantRAG("menu1.csv")

# Statik dosyaları sunma
app.mount("/static", StaticFiles(directory="static"), name="static")

@app.get("/")
async def get():
    with open("static/index.html") as f:
        return HTMLResponse(f.read())


from openai import OpenAI  # Güncellenmiş import

# Ana uygulama başlangıcında
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

@app.post("/transcribe")
async def transcribe_audio(audio: UploadFile = File(...)):
    try:
        print(f"Received audio file: {audio.filename}")
        
        with tempfile.NamedTemporaryFile(delete=False, suffix='.wav', mode='wb') as temp_audio:
            try:
                content = await audio.read()
                print(f"Read audio content length: {len(content)} bytes")
                temp_audio.write(content)
                temp_audio.flush()
                print(f"Wrote to temporary file: {temp_audio.name}")
                
                # Yeni OpenAI API syntax'ı kullanımı
                with open(temp_audio.name, "rb") as audio_file:
                    print("Starting transcription...")
                    transcript = client.audio.transcriptions.create(
                        model="whisper-1",
                        file=audio_file,
                        language="tr"
                    )
                    print("Transcription completed")
                
                return {"text": transcript.text}
                
            except Exception as inner_e:
                print(f"Error during transcription: {str(inner_e)}")
                raise inner_e
            finally:
                try:
                    os.unlink(temp_audio.name)
                    print("Temporary file cleaned up")
                except Exception as cleanup_e:
                    print(f"Error during cleanup: {str(cleanup_e)}")
                    
    except Exception as e:
        print(f"Transcription failed: {str(e)}")
        raise HTTPException(
            status_code=500,
            detail=f"Transcription failed: {str(e)}"
        )



@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await manager.connect(websocket)
    try:
        while True:
            data = await websocket.receive_json()
            message_type = data.get("type")
            message_text = data.get("text", "")
            
            if message_type == "close":
                # Sipariş kapama işlemi
                if restaurant_rag.current_order.items:
                    final_order = restaurant_rag.format_final_order()
                    await manager.send_message({
                        "type": "order_confirmation",
                        "text": "Siparişinizi onaylıyor musunuz?\n" + final_order,
                        "orderDetails": final_order,
                    }, websocket)
                else:
                    await manager.send_message({
                        "type": "close",
                        "text": "Bizi tercih ettiğiniz için teşekkürler! İyi günler!",
                    }, websocket)
                    
            elif message_type == "order_confirm":
                confirmation = data.get("confirm", False)
                if confirmation:
                    final_order = restaurant_rag.format_final_order()
                    restaurant_rag.save_order_to_file(final_order)
                    await manager.send_message({
                        "type": "order_saved",
                        "text": "Siparişiniz onaylandı ve kaydedildi.\nBizi tercih ettiğiniz için teşekkürler!",
                        "orderDetails": final_order
                    }, websocket)
                else:
                    await manager.send_message({
                        "type": "order_cancelled",
                        "text": "Sipariş iptal edildi.\nBizi tercih ettiğiniz için teşekkürler!"
                    }, websocket)
                
            elif message_type in ["voice", "text"]:
                try:
                    response = await restaurant_rag.process_message(message_text)
                    await manager.send_message(response, websocket)
                except Exception as e:
                    error_message = {
                        "text": f"Mesaj işlenirken bir hata oluştu: {str(e)}",
                        "audio": None
                    }
                    await manager.send_message(error_message, websocket)
                    
    except WebSocketDisconnect:
        manager.disconnect(websocket)
    except Exception as e:
        print(f"WebSocket error: {e}")
        try:
            error_response = {
                "text": "Bir hata oluştu. Lütfen sayfayı yenileyip tekrar deneyin.",
                "audio": None
            }
            await websocket.send_json(error_response)
        except:
            pass

nest_asyncio.apply()
import ssl

if __name__ == "__main__":
    

    uvicorn_config = uvicorn.Config(
        app=app,
        host="localhost",
        port=8443,
        ssl_certfile='/Users/gayecetindere/cert.pem',
        ssl_keyfile='/Users/gayecetindere/key.pem'
    )


    
    server = uvicorn.Server(uvicorn_config)
    server.run()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/var/folders/04/nhwvkmt13753zn37rgwn35zr0000gn/T/ipykernel_44778/290845116.py:48: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(
INFO:     Started server process [44778]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on https://localhost:8443 (Press CTRL+C to quit)


INFO:     127.0.0.1:54508 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:54508 - "GET /static/script.js?v=1 HTTP/1.1" 304 Not Modified


INFO:     ('127.0.0.1', 54534) - "WebSocket /ws" [accepted]
INFO:     connection open


Received audio file: recording.wav
Read audio content length: 4948 bytes
Wrote to temporary file: /var/folders/04/nhwvkmt13753zn37rgwn35zr0000gn/T/tmpe6cxgycf.wav
Starting transcription...
Transcription completed
Temporary file cleaned up
INFO:     127.0.0.1:57513 - "POST /transcribe HTTP/1.1" 200 OK


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful and friendly Turkish restaurant waiter. Use the following information to help the customer.

        Menu Context:
        Category: Baslangiclar
Item: Mozzarella cubuklari
Price: 115.0 TL
Ingredients: Pane mozzarella cubuklari, pesto sos

Category: Tatlilar
Item: Sufle
Price: 65.0 TL
Ingredients: Belcika cikolatasi, tereyagi, un, yumurta

Category: Ana Yemekler
Item: Antrikot
Price: 190.0 TL
Ingredients: ozel marine edilmis antrikot, kozlenmis sebzeler, patates puresi

Category: Hamburgerler
Item: Siyah Burger
Price: 95.0 TL
Ingredients: Siyah fasulye koftesi, krem p

INFO:     Shutting down
INFO:     connection closed
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [44778]


KeyboardInterrupt: 